# task1

## Define the model with/without shortcut

In [ ]:
from functools import partial
from typing import Any, Callable, List, Optional, Type, Union

import torch
import torch.nn as nn
from torch import Tensor
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter


def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
        shortcut: bool = True,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    #     mycode
        self.shortcut=shortcut

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        # mycode
        if self.shortcut:
            out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class MyResNet(nn.Module):
    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
        shortcut: bool = True,
    ) -> None:
        super().__init__()
        # _log_api_usage_once(self)
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.shortcut=shortcut # mycode
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer,
                shortcut=self.shortcut # mycode
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                    shortcut=self.shortcut # mycode
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)




## preparatioin

In [ ]:
root="./"
use_shortcut=False
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("using device ", device)

# Hyper-parameters
num_epochs = 80
learning_rate = 0.001
if use_shortcut:
    !mkdir ./logs/shortcut/
    swriter=SummaryWriter(root+"logs/shortcut/",comment=f'_shortcut_')
else:
    !mkdir ./logs/no_shortcut/
    swriter=SummaryWriter(root+"logs/no_shortcut/",comment=f'_no_shortcut_')
# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
data_root="/kaggle/input/dataset1/datase1/cifar-10-python/"
train_dataset = torchvision.datasets.CIFAR10(root=data_root,
                                                train=True,
                                                transform=transform,
                                                download=False)

test_dataset = torchvision.datasets.CIFAR10(root=data_root,
                                            train=False,
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=100,
                                            shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=100,
                                            shuffle=False)



model = MyResNet(BasicBlock, [2, 2, 2,2],num_classes=10,shortcut=use_shortcut).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler=ReduceLROnPlateau(optimizer,verbose=True)


## train

In [ ]:

# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    train_loss=0
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

        if (i + 1) % 100 == 0:
            print("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                    .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
    train_loss/=len(train_loader)
    swriter.add_scalar(tag='Loss/train_loss', scalar_value=train_loss, global_step=epoch+1)


    # Test the model
    test_loss=0
    model.eval()
    correct = 0
    total = 0    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss+=loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))
    test_loss/=len(test_loader)
    scheduler.step(test_loss)
    swriter.add_scalar(tag='Loss/test_loss', scalar_value=test_loss, global_step=epoch+1)
    swriter.add_scalar(tag='Accuracy', scalar_value=correct / total, global_step=epoch+1)

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), f'resnet_{use_shortcut}.ckpt')

# task2

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader,Dataset
import torchvision
from torchvision import transforms, datasets
from torchvision.models import GoogLeNet


import json
import matplotlib.pyplot as plt
import os
import time
import numpy as np

## data inspection

In [51]:
for r,d,f in os.walk("/kaggle/input/face-mask-detection/face_mask_detection-main/train"):
    print(len(f))
for r,d,f in os.walk("/kaggle/input/face-mask-detection/face_mask_detection-main/validation"):
    print(len(f))
for r,d,f in os.walk("/kaggle/input/face-mask-detection/face_mask_detection-main/test"):
    print(len(f))

0
60
500
0
40
300
0
90
500


## data augmentation


In [48]:
# importing PIL
from PIL import Image


class AutoAugmentDataset(Dataset):
    def __init__(self,root,transform):
        for r,d,f in os.walk(root):
            self.root=root
            self.filenames=f

            break
        self.transform=transform
        self.data=[Image.open(self.root+'/'+image) for image in self.filenames]
        self.save_root='/kaggle/working/new2/'
            
        
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self,idx):
        item=self.data[idx]
        return item
    
    def transform_save(self,round):
        for r in range(round):
            transformed=[self.transform(d) for d in self.data]
            for i,img in enumerate(transformed):
                path=self.save_root+self.filenames[i].split('.')[0]+f"_trsfm_{r}.png"
                print(path)
                img.save(path)
        for i,img in enumerate(self.data):
            path=self.save_root+self.filenames[i]
            print(path)
            img.save(path)

transform_rules=transforms.RandomChoice([
                transforms.RandomHorizontalFlip(0.7),
                transforms.RandomRotation(20),
                transforms.GaussianBlur((3,3), sigma=(0.1, 2.0)),
                transforms.Compose([transforms.Pad(10),transforms.Resize((128,128))]),
                transforms.RandomResizedCrop((128,128),scale=(0.8, 1.0)),
                transforms.RandomAdjustSharpness(1.25),
                transforms.RandomAdjustSharpness(0.75),
                transforms.RandomAutocontrast(),
            ],p=[0.7,0.7,0.3,0.5,0.6,0.2,0.2,0.3])

In [49]:
!mkdir /kaggle/working/new2/

In [ ]:
dataset = AutoAugmentDataset("/kaggle/input/face-mask-detection/face_mask_detection-main/train/without_mask",transform=transform_rules)
dataset.transform_save(5)

## train

In [12]:
class Trainer(object):
    def __init__(self):
        self.input_root="/kaggle/input/"
        self.output_root="/kaggle/working/"
        t=time.strftime('%Y-%m-%d-%H-%M-%S')
            
        # Device configuration
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print("using device ", self.device)

        # Hyper-parameters
        self.num_epochs = 100
        self.learning_rate = 0.001
        self.batch_size=64
        self.loss_weight=[0.7,0.2,0.1]
        hp_str=f"_{self.batch_size}_{'_'.join([str(f) for f in self.loss_weight])}_"

        # Image preprocessing modules
        self.transform =transforms.ToTensor()
      
        
        self.model = GoogLeNet(num_classes=2, aux_logits= True,init_weights=True).to(self.device)

        # Loss and optimizer
        self.loss_func = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.scheduler=ReduceLROnPlateau(self.optimizer,patience=5,verbose=True)
        
        self.model_save_path=self.output_root+f"models/{t+hp_str}/"
        self.log_path=self.output_root+f"logs/{t+hp_str}/"
        if not os.path.exists(self.model_save_path):
            os.makedirs(self.model_save_path)
        if not os.path.exists(self.log_path):
            os.makedirs(self.log_path)
        self.swriter=SummaryWriter(self.log_path)

    
    def load_data(self,input_root,transform,batch_size,aug=False):
        # dataset
        if aug:
            train_dataset = torchvision.datasets.ImageFolder(input_root+"face-mask-detection-aug/face_mask_detection-main/train",transform=transform)
            valid_dataset = torchvision.datasets.ImageFolder(input_root+"face-mask-detection-aug/face_mask_detection-main/validation",transform=transform)
            test_dataset = torchvision.datasets.ImageFolder(input_root+"face-mask-detection-aug/face_mask_detection-main/test",transform=transform)
        else:
            train_dataset = torchvision.datasets.ImageFolder(input_root+"face-mask-detection/face_mask_detection-main/train",transform=transform)
            valid_dataset = torchvision.datasets.ImageFolder(input_root+"face-mask-detection/face_mask_detection-main/validation",transform=transform)
            test_dataset = torchvision.datasets.ImageFolder(input_root+"face-mask-detection/face_mask_detection-main/test",transform=transform)
 
        # Data loader
        self.train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                    batch_size=batch_size,
                                                    shuffle=True)
        self.valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                                    batch_size=batch_size,
                                                    shuffle=False)
        self.test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                    batch_size=batch_size,
                                                    shuffle=False)
    
    def save_model(self,model,path):
        torch.save(model.state_dict(), path)

    def train1epoch(self):
        self.model.train()
        total_loss=0
        num=0
        data_loader=self.train_loader
        for i,(x,y) in enumerate(data_loader):
            self.optimizer.zero_grad()
            x=x.to(self.device)        
            y=y.to(self.device)

            logits,aux_logits2,aux_logits1=self.model(x)
            loss0=self.loss_func(logits,y)
            loss1=self.loss_func(aux_logits2,y)
            loss2=self.loss_func(aux_logits1,y)
            loss=loss0*self.loss_weight[0] + loss1 *self.loss_weight[1] + loss2 *self.loss_weight[2]
            loss.backward()
            self.optimizer.step()
            total_loss+=loss.item()
            num+=len(y)


        avg_loss=total_loss/num
        return avg_loss

    def calc_confuseMatrix(self,pred_y,actual_y):
        TP,FN,FP,TN=0,0,0,0
        positive=sum(pred_y)
        negative=len(pred_y)-positive
        TP_TN=sum(pred_y==actual_y)
        TP=sum(np.logical_and(pred_y==actual_y,pred_y==1))
        TN=TP_TN-TP
        FP=positive-TP
        FN=negative-TN
        return TP,FN,FP,TN
        
    def calc_GMean(self,pred_y,actual_y):
        TP,FN,FP,TN=self.calc_confuseMatrix(pred_y,actual_y)
        return np.sqrt((TP*TN)/((TP+FN)*(TN+FP)))

    def test1epoch(self,mode):
        self.model.eval()
        acc = 0.0  # accumulate accurate number / epoch
        total_loss=0
        num=0
        actual_y=[]
        pred_y=[]
        
        if mode=="valid":
            data_loader=self.valid_loader
        elif mode=="test":
            data_loader=self.test_loader
        else:
            raise Exception("!!!")
            
        with torch.no_grad():
            for x, y in data_loader:
                x=x.to(self.device)        
                y=y.to(self.device)
                outputs = self.model(x)  # eval model only have last output layer
                loss=self.loss_func(outputs,y)
                predict_y = torch.max(outputs, dim=1)[1]
                acc += (predict_y == y).sum().item()
                total_loss+=loss.item()
                num+=len(y)
                actual_y.extend(y.cpu().numpy().tolist())
                pred_y.extend(predict_y.cpu().numpy().tolist())

        acc = acc / num
        avg_loss=total_loss/num
        actual_y=np.array(actual_y)
        pred_y=np.array(pred_y)
        
#         print(actual_y)
#         print(pred_y)
        gmean=self.calc_GMean(pred_y,actual_y)
        return acc,avg_loss,gmean

    def train(self,epoch=3,loss_weight=[0.7,0.2,0.1]):
        best_acc=0

        for e in range(epoch):
            train_loss=self.train1epoch()
            self.swriter.add_scalar("Loss/train_loss",train_loss,e)
            acc,test_loss,gmean=self.test1epoch("valid")
            self.swriter.add_scalar("Loss/test_loss",test_loss,e)
            self.swriter.add_scalar("Acc",acc,e)
            self.swriter.add_scalar("GMean",gmean,e)

            self.scheduler.step(test_loss)
            if acc > best_acc:
                best_acc = acc
                if e>10:
                    self.save_model(self.model, self.model_save_path+f"EPOCH_{e}_loss_{test_loss*100}_ACC_{acc}_gmean_{gmean}.pth")
            print('[epoch %d] train_loss: %.7f valid_loss: %.7f  valid_accuracy: %.4f valid_gmean %.4f' %
                  (e + 1,train_loss*100, test_loss*100, acc,gmean))
            
    def predict(self):
        print("start inferring")
        acc,test_loss,gmean=self.test1epoch("test")
        print(f"acc {acc}")
        print(f"test_loss {test_loss}")
        print(f"gmean {gmean}")

In [5]:
trainer=Trainer()
trainer.load_data(trainer.input_root,trainer.transform,trainer.batch_size)
trainer.train(100,loss_weight=[0.7,0.2,0.1])
trainer.predict()

'''
[epoch 100] train_loss: 0.0069437 valid_loss: 0.3874206  valid_accuracy: 0.9647 valid_gmean 0.9244
acc 0.9389830508474576
test_loss 0.004772630558049275
gmean 0.9179687721631203
'''

using device  cuda
[epoch 1] train_loss: 0.5617731 valid_loss: 1.1479838  valid_accuracy: 0.8824 valid_gmean 0.0000
[epoch 2] train_loss: 0.3469810 valid_loss: 1.1241330  valid_accuracy: 0.8824 valid_gmean 0.0000
[epoch 3] train_loss: 0.2622484 valid_loss: 0.9321179  valid_accuracy: 0.7029 valid_gmean 0.8062
[epoch 4] train_loss: 0.2154961 valid_loss: 2.8175017  valid_accuracy: 0.4412 valid_gmean 0.6055
[epoch 5] train_loss: 0.1947173 valid_loss: 1.2860239  valid_accuracy: 0.9000 valid_gmean 0.4176
[epoch 6] train_loss: 0.1734504 valid_loss: 0.5788869  valid_accuracy: 0.9029 valid_gmean 0.8563
[epoch 7] train_loss: 0.1785824 valid_loss: 0.6897403  valid_accuracy: 0.9000 valid_gmean 0.4720
[epoch 8] train_loss: 0.1187753 valid_loss: 2.8497068  valid_accuracy: 0.8853 valid_gmean 0.5376
[epoch 9] train_loss: 0.2289705 valid_loss: 1.0498409  valid_accuracy: 0.9324 valid_gmean 0.7940
[epoch 10] train_loss: 0.1929488 valid_loss: 1.9177885  valid_accuracy: 0.8912 valid_gmean 0.2739
[epoch 11]

ValueError: too many values to unpack (expected 2)

In [13]:
# train on augmented data
trainer=Trainer()
trainer.load_data(trainer.input_root,trainer.transform,trainer.batch_size,True)
trainer.train(trainer.num_epochs,loss_weight=[0.7,0.2,0.1])
trainer.predict()

"""
[epoch 100] train_loss: 0.0027061 valid_loss: 0.1844178  valid_accuracy: 0.9882 valid_gmean 0.9714
start inferring
acc 0.9576271186440678
test_loss 0.003229771693379192
gmean 0.9567769971222251
"""

using device  cuda
[epoch 1] train_loss: 0.6585388 valid_loss: 2.7697771  valid_accuracy: 0.1176 valid_gmean 0.0000
[epoch 2] train_loss: 0.3799642 valid_loss: 6.6100092  valid_accuracy: 0.1176 valid_gmean 0.0000
[epoch 3] train_loss: 0.2742518 valid_loss: 4.1478069  valid_accuracy: 0.3882 valid_gmean 0.5538
[epoch 4] train_loss: 0.2488945 valid_loss: 0.6345603  valid_accuracy: 0.8441 valid_gmean 0.8246
[epoch 5] train_loss: 0.2226251 valid_loss: 4.0525582  valid_accuracy: 0.7735 valid_gmean 0.1478
[epoch 6] train_loss: 0.1619159 valid_loss: 0.9571270  valid_accuracy: 0.8941 valid_gmean 0.6197
[epoch 7] train_loss: 0.1959629 valid_loss: 0.4911936  valid_accuracy: 0.9294 valid_gmean 0.8064
[epoch 8] train_loss: 0.1621252 valid_loss: 1.6975680  valid_accuracy: 0.9235 valid_gmean 0.7000
[epoch 9] train_loss: 0.1047257 valid_loss: 0.4309923  valid_accuracy: 0.9676 valid_gmean 0.8646
[epoch 10] train_loss: 0.0883768 valid_loss: 0.7189341  valid_accuracy: 0.8882 valid_gmean 0.9345
[epoch 11]

In [22]:
trainer=Trainer()
trainer.load_data(trainer.input_root,trainer.transform,trainer.batch_size)


using device  cpu


In [18]:
pred=np.array([1,1,1,0])
y=np.array([1,0,1,0])
print(trainer.calc_confuseMatrix(pred,y))
trainer.calc_GMean(pred,y)

(2, 0, 1, 1)


0.7071067811865476

In [23]:
trainer.test1epoch("test")

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

(0.15254237288135594, 0.01193470550795733, 0.0)